In [1]:
import json
import numpy as np
import pandas as pd

In [2]:
file = "./renttherunway_final_data.json"
df = pd.read_json(file,lines=True)
df = df[['user_id','item_id','rating']]
df = df.dropna()

ValueError: Expected object or value

In [3]:
df.shape

(192462, 3)

In [4]:
df.head()

,user_id,item_id,rating
0,420272,2260466,10.0
1,273551,153475,10.0
2,360448,1063761,10.0
3,909926,126335,8.0
4,151944,616682,10.0


In [5]:
n_users = df.user_id.unique().shape[0]
n_items = df.item_id.unique().shape[0]
ratings = pd.DataFrame(np.zeros((n_users, n_items)), index = df.user_id.unique(), columns = list(df.item_id.unique()))
for row in df.itertuples():
    ratings.loc[row[1],row[2]] = row[3]
ratings = np.array(ratings)

In [6]:
ratings

array([[10.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0., 10.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0., 10., ...,  0.,  0.,  0.],
       ...,
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [7]:
print (str(n_users) + ' users')
print (str(n_items) + ' items')
sparsity = float(len(ratings.nonzero()[0]))
sparsity /= (ratings.shape[0] * ratings.shape[1])
sparsity *= 100
print ('Sparsity: {:4.2f}%'.format(sparsity))

105508 users
5850 items
Sparsity: 0.03%


In [8]:
max_count = 0
one_count = 0
two_count = 0
three_count = 0
for i in range(ratings.shape[0]):
    if len(ratings[i, :].nonzero()[0]) > 9:
        max_count+=1
        
    if len(ratings[i, :].nonzero()[0]) == 1:
        one_count += 1
        
    if len(ratings[i, :].nonzero()[0]) == 2:
        two_count += 1
        
    if len(ratings[i, :].nonzero()[0]) == 3:
        three_count += 1

In [9]:
print(one_count/n_users)
print(two_count/n_users)
print(three_count/n_users)
print(max_count)

0.6808014558137772
0.16936156499981045
0.06513250180081132
1388


In [10]:
test_users = []
for i in range(ratings.shape[0]):
    if len(ratings[i, :].nonzero()[0]) > 9:
        test_users.append(i)

In [11]:
len(test_users)

1388

In [12]:
def train_test_split(ratings, test_users, s):
    test = np.zeros(ratings.shape)
    train = ratings.copy()
    for user in test_users:
        test_ratings = np.random.choice(ratings[user, :].nonzero()[0], size=s, replace=False)
        train[user, test_ratings] = 0.
        test[user, test_ratings] = ratings[user, test_ratings]
        
    # Test and training are truly disjoint
    assert(np.all((train * test) == 0)) 
    return train, test

In [13]:
train, test = train_test_split(ratings, test_users, 5)